# Agent Registry Demo — Discovery & Metadata

This notebook demonstrates the **Microsoft Entra Agent Registry** - a centralized metadata repository and discovery service for AI agents.

**What you'll learn:**
- Register agents to the registry with metadata (skills, capabilities, protocols)
- Query the registry to discover agents
- Search by skills and capabilities
- Understand collections and discovery boundaries

**Prerequisites:**
- Completed `basic_agent_auth_demo.ipynb` (need agent identity)
- Service principal with Microsoft Graph permissions:
  - `AgentRegistry.ReadWrite.All` (to register/update agents)
  - `AgentRegistry.Read.All` (to query agents)

**References:**
- [What is Agent Registry](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/what-is-agent-registry)
- [Register agents](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/publish-agents-to-registry)
- [Agent metadata](https://learn.microsoft.com/en-us/entra/agent-id/identity-platform/agent-metadata-discoverability)

## Setup — Load Environment & Create Graph Client

In [1]:
import os
import json
import requests
from pathlib import Path
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential

# Auto-detect the examples folder
HERE = Path.cwd()
candidates = [
    HERE,
    HERE / "MICROSOFT-ENTRA-AGENT-ID" / "examples" / "basic-agent-auth",
]
BASE_DIR = next((p for p in candidates if (p / ".env").exists()), None)
if BASE_DIR is None:
    raise FileNotFoundError("Could not find .env file. Open notebook from examples folder.")

load_dotenv(BASE_DIR / ".env")

TENANT_ID = os.getenv("TENANT_ID")
CLIENT_ID = os.getenv("AGENT_CLIENT_ID")
CLIENT_SECRET = os.getenv("AGENT_CLIENT_SECRET")

if not all([TENANT_ID, CLIENT_ID, CLIENT_SECRET]):
    raise ValueError("Missing required environment variables: TENANT_ID, AGENT_CLIENT_ID, AGENT_CLIENT_SECRET")

print(f"✓ Loaded config")
print(f"  Tenant: {TENANT_ID}")
print(f"  Agent:  {CLIENT_ID}")

# Create credential for Graph API
credential = ClientSecretCredential(
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

def get_graph_token() -> str:
    """Get access token for Microsoft Graph API."""
    token_result = credential.get_token("https://graph.microsoft.com/.default")
    return token_result.token

print("✓ Graph API client ready")

✓ Loaded config
  Tenant: a172a259-b1c7-4944-b2e1-6d551f954711
  Agent:  2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
✓ Graph API client ready


## Demo Flow

1) List agent identity blueprints
2) List blueprint principals
3) List agent identities
4) List agent instances
5) Create a new blueprint + agent identity + register a full agent instance
6) Optional cleanup

In [2]:
GRAPH_BETA = "https://graph.microsoft.com/beta"
GRAPH_V1 = "https://graph.microsoft.com/v1.0"

def graph_headers():
    token = get_graph_token()
    return {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

def graph_get(url):
    return requests.get(url, headers=graph_headers(), timeout=30)

def graph_post(url, payload):
    return requests.post(url, headers=graph_headers(), json=payload, timeout=30)

def graph_delete(url):
    return requests.delete(url, headers=graph_headers(), timeout=30)

def print_list(title, items, fields):
    print(title)
    print(f"Count: {len(items)}")
    for i, item in enumerate(items, 1):
        print(f"\n{i}.")
        for field in fields:
            print(f"  {field}: {item.get(field)}")
    print()

## 1) List Agent Identity Blueprints

In [3]:
url = f"{GRAPH_BETA}/applications/microsoft.graph.agentIdentityBlueprint?$select=id,appId,displayName,createdDateTime"
response = graph_get(url)

if response.status_code == 200:
    blueprints = response.json().get("value", [])
    print_list("Agent Identity Blueprints", blueprints, ["displayName", "id", "appId", "createdDateTime"])
else:
    print(f"✗ Failed: {response.status_code}")
    print(response.text)

Agent Identity Blueprints
Count: 26

1.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 10f3baeb-c2e4-4a47-bc32-6bc936782bd5
  appId: 10f3baeb-c2e4-4a47-bc32-6bc936782bd5
  createdDateTime: 2025-12-04T21:29:30Z

2.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: d0a4316c-60d8-43f5-9a5a-e80bb7388847
  appId: d0a4316c-60d8-43f5-9a5a-e80bb7388847
  createdDateTime: 2025-12-04T21:31:41Z

3.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 12ecbd78-f028-40e8-a443-2e6b4d5aaf7e
  appId: 12ecbd78-f028-40e8-a443-2e6b4d5aaf7e
  createdDateTime: 2025-12-04T21:36:55Z

4.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 6d202dfc-d990-4659-96ba-5fab38e7d966
  appId: 6d202dfc-d990-4659-96ba-5fab38e7d966
  createdDateTime: 2025-12-04T21:37:33Z

5.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 80731f44-30d1-4701-9a6

## 2) List Blueprint Principals

In [4]:
url = f"{GRAPH_BETA}/servicePrincipals/microsoft.graph.agentIdentityBlueprintPrincipal?$select=id,appId,displayName,createdDateTime"
response = graph_get(url)

if response.status_code == 200:
    principals = response.json().get("value", [])
    print_list("Blueprint Principals", principals, ["displayName", "id", "appId", "createdDateTime"])
else:
    print(f"✗ Failed: {response.status_code}")
    print(response.text)

Blueprint Principals
Count: 22

1.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 572ae8df-2c9b-4d17-96e9-4ae90a47f0f8
  appId: 10f3baeb-c2e4-4a47-bc32-6bc936782bd5
  createdDateTime: 2025-12-04T21:29:31Z

2.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 12cc612a-12c1-42fb-a280-bc553bb9bc9d
  appId: d0a4316c-60d8-43f5-9a5a-e80bb7388847
  createdDateTime: 2025-12-04T21:31:41Z

3.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: d61f8b42-983e-4f17-bba0-5aa47ec704b1
  appId: 12ecbd78-f028-40e8-a443-2e6b4d5aaf7e
  createdDateTime: 2025-12-04T21:36:55Z

4.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 44fd75be-6a00-493d-a1c6-204e2e4e6881
  appId: 6d202dfc-d990-4659-96ba-5fab38e7d966
  createdDateTime: 2025-12-04T21:37:34Z

5.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint
  id: 6c8161b9-32c9-49ad-a596-a2d

## 3) List Agent Identities

In [5]:
url = f"{GRAPH_BETA}/servicePrincipals/microsoft.graph.agentIdentity?$select=id,displayName,agentIdentityBlueprintId,createdDateTime"
response = graph_get(url)

if response.status_code == 200:
    identities = response.json().get("value", [])
    print_list("Agent Identities", identities, ["displayName", "id", "agentIdentityBlueprintId", "createdDateTime"])
else:
    print(f"✗ Failed: {response.status_code}")
    print(response.text)

Agent Identities
Count: 16

1.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentity
  id: 38c14420-a914-4370-a0f8-1b014598c1d0
  agentIdentityBlueprintId: 10f3baeb-c2e4-4a47-bc32-6bc936782bd5
  createdDateTime: 2025-12-04T21:29:49Z

2.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentity
  id: 966ccc07-512a-4698-bafb-4d5686973d27
  agentIdentityBlueprintId: d0a4316c-60d8-43f5-9a5a-e80bb7388847
  createdDateTime: 2025-12-04T21:31:45Z

3.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentity
  id: 9350dda6-b732-4b1c-a111-c5d8c4ffc64a
  agentIdentityBlueprintId: 12ecbd78-f028-40e8-a443-2e6b4d5aaf7e
  createdDateTime: 2025-12-04T21:37:14Z

4.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentity
  id: a3be091d-da7b-4696-b0f6-b7f41f5cca84
  agentIdentityBlueprintId: 6d202dfc-d990-4659-96ba-5fab38e7d966
  createdDateTime: 2025-12-04T21:37:38Z

5.
  displayName: aq-ai-foundry-Sweden-Central-firstProject-AgentIdentity
  id: 

## 4) List Agent Instances (Registry)

In [6]:
url = f"{GRAPH_BETA}/agentRegistry/agentInstances?$select=id,displayName,agentIdentityId,agentIdentityBlueprintId,originatingStore,url,createdDateTime"
response = graph_get(url)

if response.status_code == 200:
    agents = response.json().get("value", [])
    print_list("Agent Instances", agents, ["displayName", "id", "agentIdentityId", "agentIdentityBlueprintId", "originatingStore", "url"])
else:
    print(f"✗ Failed: {response.status_code}")
    print(response.text)

Agent Instances
Count: 14

1.
  displayName: Test Agent - Minimal (20260120-094339-95a674)
  id: 26ea9a17-3319-4ea7-a2f5-4ba151451060
  agentIdentityId: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
  agentIdentityBlueprintId: None
  originatingStore: Python Test Script
  url: https://example.com/agents/test-minimal/20260120-094339-95a674

2.
  displayName: Test Agent - Minimal (20260120-095215-c7a243)
  id: dfa604f0-bf50-4ed4-aa02-f87458750df9
  agentIdentityId: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
  agentIdentityBlueprintId: None
  originatingStore: Python Test Script
  url: https://example.com/agents/test-minimal/20260120-095215-c7a243

3.
  displayName: Test Agent - Minimal (20260120-100214-bd772e)
  id: f01a1cea-ffa8-4bc3-8397-b6ac41400726
  agentIdentityId: 2c9ecb92-2756-4983-a4c6-2884d8ba3fa1
  agentIdentityBlueprintId: None
  originatingStore: Python Test Script
  url: https://example.com/agents/test-minimal/20260120-100214-bd772e

4.
  displayName: Test Agent - Minimal (20260120-100357

## 4b) Search Agents (By Display Name)

In [10]:
search_term = "Demo"

url = f"{GRAPH_BETA}/agentRegistry/agentInstances?$filter=contains(displayName,'{search_term}')&$select=id,displayName,originatingStore,url"
response = graph_get(url)

if response.status_code == 200:
    matches = response.json().get("value", [])
    print_list(f"Agents matching '{search_term}'", matches, ["displayName", "id", "originatingStore", "url"])
elif response.status_code == 400 and "not supported" in response.text.lower():
    print("⚠️ $filter not supported. Falling back to client-side filtering.")
    all_url = f"{GRAPH_BETA}/agentRegistry/agentInstances?$select=id,displayName,originatingStore,url"
    all_resp = graph_get(all_url)
    if all_resp.status_code == 200:
        agents = all_resp.json().get("value", [])
        matches = [a for a in agents if search_term.lower() in (a.get("displayName") or "").lower()]
        print_list(f"Agents matching '{search_term}' (client-side)", matches, ["displayName", "id", "originatingStore", "url"])
    else:
        print(f"✗ Failed: {all_resp.status_code}")
        print(all_resp.text)
else:
    print(f"✗ Failed: {response.status_code}")
    print(response.text)

⚠️ $filter not supported. Falling back to client-side filtering.
Agents matching 'Demo' (client-side)
Count: 2

1.
  displayName: Demo Agent Instance 20260120-104556-6c1aaa
  id: b9f3b116-dd1c-42e5-8a0a-438eafecd594
  originatingStore: Notebook Demo
  url: https://example.com/agents/demo

2.
  displayName: Demo Agent Instance 20260120-104713-11575f
  id: 8b238e6f-3c62-4dfa-b21a-e127d4533d82
  originatingStore: Notebook Demo
  url: https://example.com/agents/demo



## 5) Create Blueprint → Agent Identity → Full Agent Instance

**Requires:**
- USER_ID in .env
- Admin role + Graph permissions for blueprint creation
- Standard app has AgentInstance.ReadWrite.All + AgentCardManifest.ReadWrite.All

In [8]:
import time
import uuid

USER_ID = os.getenv("USER_ID")
CREATED_BLUEPRINT_ID = None
CREATED_AGENT_IDENTITY_ID = None
CREATED_AGENT_INSTANCE_ID = None

if not USER_ID:
    print("⚠️ USER_ID not set. Skipping blueprint flow.")
else:
    run_id = f"{time.strftime('%Y%m%d-%H%M%S')}-{uuid.uuid4().hex[:6]}"
    print(f"Run ID: {run_id}")

    # 1) Create blueprint
    blueprint_payload = {
        "displayName": f"Demo Blueprint {run_id}",
        "description": "Notebook demo blueprint",
        "sponsors@odata.bind": [f"{GRAPH_V1}/users/{USER_ID}"],
    }
    bp_url = f"{GRAPH_BETA}/applications/microsoft.graph.agentIdentityBlueprint"
    bp_resp = graph_post(bp_url, blueprint_payload)
    if bp_resp.status_code != 201:
        print(f"✗ Blueprint create failed: {bp_resp.status_code}")
        print(bp_resp.text)
    else:
        bp = bp_resp.json()
        blueprint_object_id = bp.get("id")
        blueprint_app_id = bp.get("appId")
        CREATED_BLUEPRINT_ID = blueprint_object_id
        print(f"✓ Blueprint created: {blueprint_object_id}")

        # 2) Wait for propagation
        for attempt in range(1, 9):
            check_url = f"{GRAPH_BETA}/applications/{blueprint_object_id}/microsoft.graph.agentIdentityBlueprint?$select=id,appId"
            check = graph_get(check_url)
            if check.status_code == 200 and check.json().get("appId"):
                print("✓ Blueprint ready")
                break
            wait_s = 3 * attempt
            print(f"⏳ Waiting for propagation ({attempt}/8)...")
            time.sleep(wait_s)

        # 3) Create blueprint principal (with retry)
        principal_url = f"{GRAPH_BETA}/servicePrincipals/microsoft.graph.agentIdentityBlueprintPrincipal"
        principal_payload = {"appId": blueprint_app_id}
        principal_id = None
        for attempt in range(1, 9):
            pr_resp = graph_post(principal_url, principal_payload)
            if pr_resp.status_code == 201:
                principal_id = pr_resp.json().get("id")
                print(f"✓ Blueprint principal created: {principal_id}")
                break
            if pr_resp.status_code == 409:
                lookup = graph_get(
                    f"{GRAPH_BETA}/servicePrincipals?$filter=appId%20eq%20'{blueprint_app_id}'&$select=id"
                )
                if lookup.status_code == 200 and lookup.json().get("value"):
                    principal_id = lookup.json()["value"][0].get("id")
                    print(f"✓ Blueprint principal resolved: {principal_id}")
                break
            if pr_resp.status_code == 400:
                msg = pr_resp.json().get("error", {}).get("message", "")
                if "valid application" in msg or "NoBackingApplicationObject" in msg:
                    wait_s = 5 * attempt
                    print(f"⏳ Principal not ready yet ({attempt}/8). Waiting {wait_s}s...")
                    time.sleep(wait_s)
                    continue
            print(f"✗ Principal create failed: {pr_resp.status_code}")
            print(pr_resp.text)
            break

        # 4) Add secret (blueprint addPassword, fallback to standard addPassword)
        secret_payload = {"passwordCredential": {"displayName": "Notebook Demo Secret"}}
        secret_url = f"{GRAPH_BETA}/applications/{blueprint_object_id}/microsoft.graph.agentIdentityBlueprint/addPassword"
        secret_resp = graph_post(secret_url, secret_payload)
        if secret_resp.status_code != 200:
            secret_url = f"{GRAPH_BETA}/applications/{blueprint_object_id}/addPassword"
            secret_resp = graph_post(secret_url, secret_payload)
        if secret_resp.status_code != 200:
            print(f"✗ Secret create failed: {secret_resp.status_code}")
            print(secret_resp.text)
        else:
            secret_text = secret_resp.json().get("secretText")
            print(f"✓ Secret created (len={len(secret_text) if secret_text else 0})")
            time.sleep(30)

            # 5) Authenticate as blueprint and create agent identity
            blueprint_cred = ClientSecretCredential(
                tenant_id=TENANT_ID,
                client_id=blueprint_app_id,
                client_secret=secret_text,
            )
            blueprint_token = blueprint_cred.get_token("https://graph.microsoft.com/.default").token
            blueprint_headers = {"Authorization": f"Bearer {blueprint_token}", "Content-Type": "application/json"}
            ai_payload = {
                "displayName": f"Demo Agent Identity {run_id}",
                "agentIdentityBlueprintId": blueprint_app_id,
                "sponsors@odata.bind": [f"{GRAPH_V1}/users/{USER_ID}"],
            }
            ai_url = f"{GRAPH_BETA}/servicePrincipals/microsoft.graph.agentIdentity"
            ai_resp = requests.post(ai_url, headers=blueprint_headers, json=ai_payload, timeout=30)
            if ai_resp.status_code != 201:
                print(f"✗ Agent identity create failed: {ai_resp.status_code}")
                print(ai_resp.text)
            else:
                ai = ai_resp.json()
                CREATED_AGENT_IDENTITY_ID = ai.get("id")
                print(f"✓ Agent identity created: {CREATED_AGENT_IDENTITY_ID}")

                # 6) Register agent instance with full manifest using standard app token
                agent_payload = {
                    "displayName": f"Demo Agent Instance {run_id}",
                    "sourceAgentId": f"demo-agent-{CLIENT_ID[:8]}-{run_id}",
                    "originatingStore": "Notebook Demo",
                    "url": "https://example.com/agents/demo",
                    "preferredTransport": "JSONRPC",
                    "managedBy": CLIENT_ID,
                    "ownerIds": [USER_ID],
                    "agentIdentityBlueprintId": blueprint_object_id,
                    "agentIdentityId": CREATED_AGENT_IDENTITY_ID,
                    "agentCardManifest": {
                        "displayName": "Demo Agent Card",
                        "description": "Notebook demo agent card",
                        "protocolVersion": "1.0",
                        "version": "1.0.0",
                        "capabilities": {
                            "streaming": False,
                            "pushNotifications": False,
                            "stateTransitionHistory": False,
                            "extensions": [],
                        },
                        "supportsAuthenticatedExtendedCard": False,
                        "skills": [
                            {
                                "id": "demo-skill-1",
                                "displayName": "Demo Skill 1",
                                "description": "Demonstrates a skill entry",
                                "examples": ["Example task"],
                                "tags": ["demo"]
                            }
                        ],
                        "defaultInputModes": ["application/json"],
                        "defaultOutputModes": ["application/json"],
                        "provider": {"organization": "Demo Org", "url": "https://example.com"}
                    }
                }
                reg_url = f"{GRAPH_BETA}/agentRegistry/agentInstances"
                reg_resp = graph_post(reg_url, agent_payload)
                if reg_resp.status_code == 201:
                    CREATED_AGENT_INSTANCE_ID = reg_resp.json().get("id")
                    print(f"✓ Agent instance created: {CREATED_AGENT_INSTANCE_ID}")
                else:
                    print(f"✗ Agent instance create failed: {reg_resp.status_code}")
                    print(reg_resp.text)

Run ID: 20260120-104713-11575f
✓ Blueprint created: 74b5daa7-e192-4c20-bc4c-66b5587f5f00
⏳ Waiting for propagation (1/8)...
⏳ Waiting for propagation (2/8)...
✓ Blueprint ready
✓ Blueprint principal created: 93008ca4-6a6f-4eb8-bd1a-8cd27bfd15a3
✓ Secret created (len=40)
✓ Agent identity created: 3978890a-8e7f-4086-b6e3-47f49fce57c1
✓ Agent instance created: 8b238e6f-3c62-4dfa-b21a-e127d4533d82


## 6) Selective Delete (Pick IDs)

Set one or more IDs below to delete specific objects. Leave as None to skip.

In [11]:
# Quick ID reference (latest lists)
def print_id_table(title, items, id_field="id", name_field="displayName"):
    print(title)
    for item in items:
        print(f"- {item.get(name_field)} :: {item.get(id_field)}")
    print()

# Blueprints
bp_resp = graph_get(f"{GRAPH_BETA}/applications/microsoft.graph.agentIdentityBlueprint?$select=id,displayName")
if bp_resp.status_code == 200:
    print_id_table("Blueprints", bp_resp.json().get("value", []))

# Blueprint principals
pr_resp = graph_get(f"{GRAPH_BETA}/servicePrincipals/microsoft.graph.agentIdentityBlueprintPrincipal?$select=id,displayName")
if pr_resp.status_code == 200:
    print_id_table("Blueprint Principals", pr_resp.json().get("value", []))

# Agent identities
ai_resp = graph_get(f"{GRAPH_BETA}/servicePrincipals/microsoft.graph.agentIdentity?$select=id,displayName")
if ai_resp.status_code == 200:
    print_id_table("Agent Identities", ai_resp.json().get("value", []))

# Agent instances
ag_resp = graph_get(f"{GRAPH_BETA}/agentRegistry/agentInstances?$select=id,displayName")
if ag_resp.status_code == 200:
    print_id_table("Agent Instances", ag_resp.json().get("value", []))

# Recently created (if available in this session)
print("Recently created (this session):")
print(f"- Blueprint: {CREATED_BLUEPRINT_ID}")
print(f"- Agent identity: {CREATED_AGENT_IDENTITY_ID}")
print(f"- Agent instance: {CREATED_AGENT_INSTANCE_ID}")

Blueprints
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 10f3baeb-c2e4-4a47-bc32-6bc936782bd5
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: d0a4316c-60d8-43f5-9a5a-e80bb7388847
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 12ecbd78-f028-40e8-a443-2e6b4d5aaf7e
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 6d202dfc-d990-4659-96ba-5fab38e7d966
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 80731f44-30d1-4701-9a66-f37e944625e5
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 503ace0c-2bb8-4982-9879-2d0c84d3b122
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 11e05fb4-3b78-4017-bd31-29dfc0c7c24d
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 321a067b-1783-403d-ab9b-13838c9e38ec
- aq-ai-foundry-Sweden-Central-firstProject-AgentIdentityBlueprint :: 3f9eab96-b40c-49d1-ad5c-14ec05ec50ba
- r2d2-foundry-001-Main-Pr

In [13]:
# Set IDs to delete (object IDs). Leave None to skip.
DELETE_AGENT_INSTANCE_ID = None
DELETE_AGENT_IDENTITY_ID = None
DELETE_BLUEPRINT_ID = None
DELETE_BLUEPRINT_PRINCIPAL_ID = None

def try_delete(label, url, obj_id):
    if not obj_id:
        print(f"- Skipping {label} (no ID)")
        return
    resp = graph_delete(url)
    print(f"- {label} {obj_id}: {resp.status_code}")
    if resp.status_code not in [200, 204]:
        print(resp.text)

print("Selective delete:")

if DELETE_AGENT_INSTANCE_ID:
    try_delete("Agent instance", f"{GRAPH_BETA}/agentRegistry/agentInstances/{DELETE_AGENT_INSTANCE_ID}", DELETE_AGENT_INSTANCE_ID)
else:
    print("- Skipping agent instance (no ID)")

if DELETE_AGENT_IDENTITY_ID:
    try_delete("Agent identity", f"{GRAPH_BETA}/servicePrincipals/{DELETE_AGENT_IDENTITY_ID}", DELETE_AGENT_IDENTITY_ID)
else:
    print("- Skipping agent identity (no ID)")

if DELETE_BLUEPRINT_PRINCIPAL_ID:
    try_delete("Blueprint principal", f"{GRAPH_BETA}/servicePrincipals/{DELETE_BLUEPRINT_PRINCIPAL_ID}", DELETE_BLUEPRINT_PRINCIPAL_ID)
else:
    print("- Skipping blueprint principal (no ID)")

if DELETE_BLUEPRINT_ID:
    try_delete("Blueprint application", f"{GRAPH_BETA}/applications/{DELETE_BLUEPRINT_ID}", DELETE_BLUEPRINT_ID)
else:
    print("- Skipping blueprint application (no ID)")

Selective delete:
- Skipping agent instance (no ID)
- Skipping agent identity (no ID)
- Skipping blueprint principal (no ID)
- Skipping blueprint application (no ID)


## 7) Cleanup (Optional)

In [14]:
# Set RUN_CLEANUP = True to delete objects created in step 5
RUN_CLEANUP = False

if RUN_CLEANUP:
    if CREATED_AGENT_INSTANCE_ID:
        del_url = f"{GRAPH_BETA}/agentRegistry/agentInstances/{CREATED_AGENT_INSTANCE_ID}"
        del_resp = graph_delete(del_url)
        print(f"Delete agent instance: {del_resp.status_code}")
    if CREATED_BLUEPRINT_ID:
        del_url = f"{GRAPH_BETA}/applications/{CREATED_BLUEPRINT_ID}"
        del_resp = graph_delete(del_url)
        print(f"Delete blueprint: {del_resp.status_code}")
else:
    print("Cleanup skipped. Set RUN_CLEANUP = True to delete created objects.")

Cleanup skipped. Set RUN_CLEANUP = True to delete created objects.
